# Bài tập 1. 
Minh họa thuật toán chuyển biểu thức dạng trung tố sang dạng hậu tố với một vài toán tử cơ bản (sin, có, sqrt, !,-, +, *, %. ? ...)

# Tư duy Triển khai: Chuyển đổi Infix sang Postfix (Shunting-yard)

## 1. Mục tiêu bài toán
Chuyển biểu thức toán học từ dạng con người đọc (Trung tố - `A + B`) sang dạng máy tính tính toán (Hậu tố - `A B +`) bằng cách sử dụng **Stack**.

* Biểu thức trung tố (Infix): A + B * C
* Con người hiểu: Nhân chia trước, cộng trừ sau ($B \times C$ rồi mới $+ A$).
* Máy tính đọc từ trái sang phải: Gặp A + B, nó muốn cộng ngay. Nhưng nó phải chờ xem phía sau có phép nhân chia nào "mạnh" hơn không.
* Giải pháp:Chúng ta cần một "nhà chờ" cho các toán tử.
    - Số (Toán hạng): Cho ra kết quả ngay (Output Queue).
    - Dấu (Toán tử): Đưa vào "nhà chờ" (Stack). Dấu nào mạnh hơn được ưu tiên xử lý trước.

## 2. Cấu trúc dữ liệu cần thiết
* **Input:** Chuỗi biểu thức gốc (String).
* **Output Queue (List):** Danh sách chứa kết quả hậu tố (các số và toán tử đã được sắp xếp).
* **Operator Stack (Stack):** "Nhà chờ" tạm thời cho các toán tử (`+`, `-`, `*`, `sin`, `(`...) khi chưa đến lượt xử lý.
* **Precedence Map (Dict):** Bảng tra cứu độ ưu tiên (độ mạnh) của toán tử.

## 3. Bảng độ ưu tiên (Precedence Rules)
Quy ước điểm số để so sánh:
* `4`: Hàm số (`sin`, `cos`, `sqrt`, `!`)
* `3`: Lũy thừa (`^`)
* `2`: Nhân, Chia, Dư (`*`, `/`, `%`)
* `1`: Cộng, Trừ (`+`, `-`)
* `0`: Ngoặc mở (`(`) - *Thấp nhất để không bị pop nhầm*.

## 4. Luồng xử lý chính (Logic Flow)

**Bước 0: Tiền xử lý (Tokenization)**
* Dùng Regex để tách chuỗi input thành các token riêng biệt.
* Ví dụ: `sin(x)+1` -> `['sin', '(', 'x', ')', '+', '1']`.

**Bước 1: Duyệt từng Token từ trái sang phải**

### A. Nếu Token là SỐ hoặc BIẾN (Operand)
* **Hành động:** Đẩy thẳng vào `Output Queue`.
* *Tư duy:* Số hạng không cần xếp hàng, đi ra luôn.

### B. Nếu Token là HÀM (`sin`, `sqrt`...) hoặc `(`
* **Hành động:** Đẩy (Push) vào `Operator Stack`.
* *Tư duy:* Đánh dấu bắt đầu một vùng ưu tiên mới.

### C. Nếu Token là `)` (Đóng ngoặc)
* **Hành động:**
    1.  Lặp: Pop liên tục các toán tử từ `Stack` sang `Output` cho đến khi gặp `(`.
    2.  Pop và loại bỏ `(`.
    3.  **Quan trọng:** Kiểm tra đỉnh Stack ngay sau đó. Nếu là **Hàm** (`sin`, `cos`...) -> Pop nốt hàm đó sang `Output`.
* *Tư duy:* Giải quyết xong nội dung trong ngoặc (hoặc trong hàm), gom tất cả về kết quả.

### D. Nếu Token là TOÁN TỬ (`+`, `-`, `*`, `/`...)
* **Hành động:**
    1.  So sánh độ ưu tiên của `Token hiện tại` với `Đỉnh Stack`.
    2.  **Vòng lặp (While):** Chừng nào `Stack còn phần tử` VÀ `Độ ưu tiên Đỉnh Stack >= Token hiện tại`:
        * Pop `Đỉnh Stack` sang `Output`.
    3.  Sau khi dọn dẹp xong những kẻ mạnh hơn mình, Push `Token hiện tại` vào `Stack`.
* *Tư duy:* "Kẻ yếu đến sau phải đợi kẻ mạnh đi trước".

**Bước 2: Xử lý tàn dư**
* Sau khi duyệt hết chuỗi Input: Pop toàn bộ các toán tử còn sót lại trong `Stack` sang `Output`.

## 5. Ví dụ chạy tay (Trace)
Input: `3 + sin(x) * 2`

| Token | Stack (Nhà chờ) | Output (Kết quả) | Giải thích |
| :--- | :--- | :--- | :--- |
| `3` | `[]` | `3` | Số ra luôn |
| `+` | `[+]` | `3` | Stack rỗng, vào luôn |
| `sin` | `[+, sin]` | `3` | Hàm ưu tiên cao, vào đè lên + |
| `(` | `[+, sin, (]` | `3` | Ngoặc mở, vào luôn |
| `x` | `[+, sin, (]` | `3 x` | Biến ra luôn |
| `)` | `[+, sin]` | `3 x` | Gặp đóng ngoặc, pop đến `(` |
| *(Check hàm)* | `[+]` | `3 x sin` | Thấy `sin` trước ngoặc, pop luôn `sin` |
| `*` | `[+, *]` | `3 x sin` | `*` mạnh hơn `+`, được phép vào đè lên |
| `2` | `[+, *]` | `3 x sin 2` | Số ra luôn |
| **End** | `[]` | `3 x sin 2 * +` | Pop hết Stack |

**Kết quả:** `3 x sin 2 * +`

In [1]:
import re

def infix_to_postfix(expression):
    # 1. Định nghĩa độ ưu tiên
    precedence = {
        'sin': 4, 'cos': 4, 'sqrt': 4, '!': 4,
        '^': 3,
        '*': 2, '/': 2, '%': 2,
        '+': 1, '-': 1,
        '(': 0
    }
    
    # 2. Xử lý chuỗi đầu vào (Tokenization)
    # Tách số (gồm cả thập phân), chữ (tên hàm/biến), và các dấu
    # Pattern giải thích: (\d+\.?\d*) là số, ([a-z]+) là chữ, (.) là các ký tự đơn lẻ còn lại
    tokens = re.findall(r'(\d+\.?\d*|[a-z]+|.)', expression.replace(" ", ""))
    
    output_queue = []
    operator_stack = []
    
    # Hàm kiểm tra xem token có phải là số thực không
    def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            return False

    # 3. Duyệt qua từng token
    for token in tokens:
        
        # TRƯỜNG HỢP 1: Là số
        if is_number(token):
            output_queue.append(token)
            
        # TRƯỜNG HỢP 2: Là tên biến (x, y...) nhưng không phải tên hàm (sin, cos...)
        elif token.isalpha() and token not in precedence: 
            output_queue.append(token)
            
        # TRƯỜNG HỢP 3: Là hàm số (sin, cos...) hoặc Ngoặc mở
        elif token in ['sin', 'cos', 'sqrt', '(']:
            operator_stack.append(token)
            
        # TRƯỜNG HỢP 4: Là Ngoặc đóng
        elif token == ')':
            top_token = operator_stack.pop()
            while top_token != '(':
                output_queue.append(top_token)
                if not operator_stack:
                    raise ValueError("Mismatched parentheses")
                top_token = operator_stack.pop()
            
            # Sau khi pop '(', kiểm tra xem trước ngoặc có phải là hàm không (ví dụ: sin(...))
            if operator_stack and operator_stack[-1] in ['sin', 'cos', 'sqrt']:
                output_queue.append(operator_stack.pop())
                
        # TRƯỜNG HỢP 5: Là Toán tử (+, -, *, /, ^, !, %)
        elif token in precedence:
            curr_op_val = precedence[token]
            
            # Logic: Chừng nào đỉnh Stack còn mạnh hơn hoặc bằng token hiện tại -> Đuổi đỉnh Stack ra
            while (operator_stack and 
                   operator_stack[-1] in precedence and 
                   precedence[operator_stack[-1]] >= curr_op_val):
                
                # Lưu ý nhỏ: Phép lũy thừa '^' thường kết hợp phải (Right Associative)
                # nên nếu gặp '^' và đỉnh cũng là '^' thì KHÔNG pop. 
                # Ở đây ta làm phiên bản đơn giản (Left Associative) cho dễ hiểu.
                
                output_queue.append(operator_stack.pop())
            
            operator_stack.append(token)
        
        # Các ký tự rác (nếu có)
        else:
            continue

    # 4. Sau khi duyệt hết, Pop tất cả những gì còn lại trong Stack
    while operator_stack:
        op = operator_stack.pop()
        if op == '(':
            raise ValueError("Mismatched parentheses")
        output_queue.append(op)
        
    return " ".join(output_queue)

# --- CHẠY THỬ ---
expressions = [
    "3 + 4",
    "3 + 4 * 2 / ( 1 - 5 )",
    "sin( 30 ) + sqrt( 4 ) * 2",
    "10 + 2 * 3 ^ 2"
]

print(f"{'TRUNG TỐ (INFIX)':<30} | {'HẬU TỐ (POSTFIX)':<30}")
print("-" * 65)
for exp in expressions:
    postfix = infix_to_postfix(exp)
    print(f"{exp:<30} | {postfix:<30}")

TRUNG TỐ (INFIX)               | HẬU TỐ (POSTFIX)              
-----------------------------------------------------------------
3 + 4                          | 3 4 +                         
3 + 4 * 2 / ( 1 - 5 )          | 3 4 2 * 1 5 - / +             
sin( 30 ) + sqrt( 4 ) * 2      | 30 sin 4 sqrt 2 * +           
10 + 2 * 3 ^ 2                 | 10 2 3 2 ^ * +                
